In [22]:
import pandas as pd
import numpy as np
import math
pd.set_option('display.max_colwidth', None)

In [ ]:
print('ACCOUNTS BALANCE')
pd.read_csv('avk_hackathon_data_account_x_balance.csv').head(15)

In [ ]:
print('TINKOFF PRODUCTS PER PERSON')
pd.read_csv('avk_hackathon_data_party_products.csv').head(15)

In [ ]:
print('SOCIAL')
pd.read_csv('avk_hackathon_data_party_x_socdem.csv').head(15)

In [ ]:
print('STORIES')
pd.read_csv('avk_hackathon_data_story_logs.csv').head(15)

In [ ]:
print('STORIES TEXTS')
pd.read_csv('avk_hackathon_data_story_texts.csv').head(15)

In [55]:
print('TRANSACTIONS')
raw_transactions = pd.read_csv('avk_hackathon_data_transactions.csv')#nrows=30000)
raw_transactions.head()

TRANSACTIONS


,party_rk,account_rk,financial_account_type_cd,transaction_dttm,transaction_type_desc,transaction_amt_rur,merchant_rk,merchant_type,merchant_group_rk,category
0,20337,19666,1,2019-01-01,Покупка,84.00,88676.0,348.0,NaN,Сувениры
1,63404,72991,1,2019-01-01,Покупка,410.00,887248.0,330.0,725.0,Фаст Фуд
2,24789,23517,2,2019-01-01,Покупка,701.44,830014.0,291.0,NaN,Супермаркеты
3,57970,64838,2,2019-01-01,Покупка,6203.70,363834.0,278.0,454.0,Дом/Ремонт
4,12232,11591,2,2019-01-01,Покупка,734.53,85919.0,286.0,878.0,Супермаркеты


In [ ]:
raw_transactions.describe()

<h2>(Non-working draft) Artificial Geolocation

In [ ]:
merchant_features = ['merchant_rk', 'merchant_type', 'merchant_group_rk', 'category']
merchants_data = raw_transactions[merchant_features]

In [ ]:
grouped_mrk = merchants_data.groupby('merchant_rk')
grouped_mrk = grouped_mrk.agg({"category": "nunique"})
grouped_mrk = grouped_mrk.reset_index()
#grouped_mrk[grouped_mrk['category'] > 1]

In [ ]:
merchants_data.drop_duplicates(subset=['merchant_rk'])
merchants_geoloc = merchants_data
moscow_lat, moscow_long = 55.75167, 37.61778
radial_lat, radial_long = 58.307180, 33.793385
radius = math.sqrt((moscow_lat - radial_lat)**2 + (moscow_long - radial_long)**2)
geo_stddev = radius / 3
geo_lattitude = np.random.normal(moscow_lat, geo_stddev, merchants_data.shape[0])
geo_longitude = np.random.normal(moscow_long, geo_stddev, merchants_data.shape[0])
merchants_geoloc.assign(**{'lattitude': geo_lattitude, 'longitude': geo_longitude})
#merchants_geoloc

<h2>Flagged categories per party_rk based on transactions

In [71]:
unique_categories = raw_transactions['category'].drop_duplicates()
parties_activity_months = raw_transactions[['party_rk', 'transaction_amt_rur', 'category']].copy()
parties_activity_months.insert(loc = 1, column='month', value=pd.to_datetime(raw_transactions['transaction_dttm']).dt.month)
#parties_activity_months.insert(loc = 2, column='year', value=pd.to_datetime(raw_transactions['transaction_dttm']).dt.year)
#parties_activity_months.drop('transaction_dttm', axis=1, inplace=True)
print(parties_activity_months.category.unique())
print(parties_activity_months)
parties_activity_months = pd.get_dummies(parties_activity_months, columns=['month', 'category']) # 'year'
months_labels = parties_activity_months.loc[:, parties_activity_months.columns.str.startswith('month')];
# years_labels = parties_activity_months.loc[:, parties_activity_months.columns.str.startswith('year')];
parties_activity_months.shape

['Сувениры' 'Фаст Фуд' 'Супермаркеты' 'Дом/Ремонт' 'Сервисные услуги'
 'Красота' nan 'Разные товары' 'Транспорт' 'Медицинские услуги' 'Топливо'
 'Одежда/Обувь' 'Наличные' 'Связь/Телеком' 'Частные услуги'
 'Финансовые услуги' 'Рестораны' 'Развлечения' 'НКО' 'Книги' 'Кино'
 'Автоуслуги' 'Музыка' 'Отели' 'Аптеки' 'Цветы' 'Ж/д билеты' 'Авиабилеты'
 'Спорттовары' 'Госсборы' 'Аренда авто' 'Животные' 'Duty Free'
 'Турагентства' 'Образование' 'Искусство' 'Фото/Видео']
          party_rk  month  transaction_amt_rur      category
0            20337      1                84.00      Сувениры
1            63404      1               410.00      Фаст Фуд
2            24789      1               701.44  Супермаркеты
3            57970      1              6203.70    Дом/Ремонт
4            12232      1               734.53  Супермаркеты
...            ...    ...                  ...           ...
11987612     44990     12               121.00      Фаст Фуд
11987613     53023     12               102.29 

(11987617, 50)

In [72]:
parties_activity_months = parties_activity_months.groupby(['party_rk', *months_labels, *years_labels], as_index=False).sum()
parties_activity_months.drop(['party_rk', 'transaction_amt_rur', 'category_Супермаркеты', 'category_Разные товары'], axis=1, inplace=True)
categories_labels = parties_activity_months.columns.str.startswith('category_')
categories_labels = parties_activity_months.columns[categories_labels]
parties_activity_months[categories_labels] = parties_activity_months[categories_labels] > 0.0
parties_activity_months

,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,...,category_Спорттовары,category_Сувениры,category_Топливо,category_Транспорт,category_Турагентства,category_Фаст Фуд,category_Финансовые услуги,category_Фото/Видео,category_Цветы,category_Частные услуги
0,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
1,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
2,0,0,0,0,0,0,0,0,1,0,...,False,False,False,False,False,False,True,False,False,False
3,0,0,0,0,0,0,0,1,0,0,...,False,False,False,False,False,False,False,False,False,False
4,0,0,0,0,0,0,1,0,0,0,...,False,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461730,0,1,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,True,False,False,False
461731,1,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
461732,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False
461733,0,0,0,0,0,0,0,0,0,0,...,False,False,False,False,False,False,False,False,False,False


<h3> Mining association rules from the data above

In [73]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

frequent_itemsets = apriori(parties_activity_months, min_support=0.07, use_colnames=True)

In [74]:
def sort_by_metric(metric, threshold):
    rules = association_rules(frequent_itemsets, metric=metric, min_threshold=threshold)
    not_obvious_rules = ~(rules['consequents'].astype(str).str.count(',') == 0 & (rules['consequents'].astype(str).str.contains('year_') | rules['consequents'].astype(str).str.contains('month_')))
    rules = rules[not_obvious_rules]
    by_metric = rules.sort_values(metric, ascending=False)[['antecedents', 'consequents', metric]].head(1000)
    by_metric.to_csv(metric + '.csv', index=False, encoding='utf-8')
    return by_metric

In [76]:
sort_by_metric('support', 0.1)

,antecedents,consequents,support
107,(category_Транспорт),"(category_Финансовые услуги, category_Фаст Фуд)",0.132013
106,(category_Фаст Фуд),"(category_Финансовые услуги, category_Транспорт)",0.132013
105,(category_Финансовые услуги),"(category_Фаст Фуд, category_Транспорт)",0.132013
76,(category_Фаст Фуд),"(category_Финансовые услуги, category_Аптеки)",0.126832
77,(category_Аптеки),"(category_Финансовые услуги, category_Фаст Фуд)",0.126832
75,(category_Финансовые услуги),"(category_Аптеки, category_Фаст Фуд)",0.126832
93,(category_Рестораны),"(category_Финансовые услуги, category_Фаст Фуд)",0.124838
95,(category_Фаст Фуд),"(category_Рестораны, category_Финансовые услуги)",0.124838
94,(category_Финансовые услуги),"(category_Рестораны, category_Фаст Фуд)",0.124838
87,(category_Рестораны),"(category_Фаст Фуд, category_Транспорт)",0.117292


In [77]:
sort_by_metric('confidence', 0.5)

,antecedents,consequents,confidence
137,"(category_Рестораны, category_Транспорт)","(category_Финансовые услуги, category_Фаст Фуд)",0.613512
131,"(category_Аптеки, category_Транспорт)","(category_Финансовые услуги, category_Фаст Фуд)",0.573897
126,"(category_Рестораны, category_Аптеки)","(category_Финансовые услуги, category_Фаст Фуд)",0.568734
119,"(category_Рестораны, category_Аптеки)","(category_Фаст Фуд, category_Транспорт)",0.549858
120,"(category_Рестораны, category_Транспорт)","(category_Аптеки, category_Фаст Фуд)",0.542500
121,"(category_Аптеки, category_Транспорт)","(category_Рестораны, category_Фаст Фуд)",0.541477
136,"(category_Рестораны, category_Финансовые услуги)","(category_Фаст Фуд, category_Транспорт)",0.521497
107,(category_Рестораны),"(category_Финансовые услуги, category_Фаст Фуд)",0.510364
39,(category_Красота),"(category_Аптеки, category_Фаст Фуд)",0.502903
114,(category_Транспорт),"(category_Финансовые услуги, category_Фаст Фуд)",0.500632


In [78]:
sort_by_metric('lift', 1)

,antecedents,consequents,lift
307,"(category_Рестораны, category_Фаст Фуд)","(category_Аптеки, category_Транспорт)",2.825294
310,"(category_Аптеки, category_Транспорт)","(category_Рестораны, category_Фаст Фуд)",2.825294
306,"(category_Рестораны, category_Аптеки)","(category_Фаст Фуд, category_Транспорт)",2.778503
311,"(category_Фаст Фуд, category_Транспорт)","(category_Рестораны, category_Аптеки)",2.778503
309,"(category_Аптеки, category_Фаст Фуд)","(category_Рестораны, category_Транспорт)",2.714647
...,...,...,...
131,(category_Финансовые услуги),"(category_Аптеки, category_Одежда/Обувь)",1.130434
221,(category_Финансовые услуги),"(category_Наличные, category_Фаст Фуд)",1.125152
161,(category_Финансовые услуги),"(category_Аптеки, category_Топливо)",1.122957
101,(category_Финансовые услуги),"(category_Аптеки, category_Дом/Ремонт)",1.115616


In [82]:
sort_by_metric('leverage', 0)

,antecedents,consequents,leverage
269,(category_Рестораны),"(category_Фаст Фуд, category_Транспорт)",0.068886
301,(category_Транспорт),"(category_Финансовые услуги, category_Фаст Фуд)",0.067662
271,(category_Транспорт),"(category_Рестораны, category_Фаст Фуд)",0.066755
281,(category_Рестораны),"(category_Финансовые услуги, category_Фаст Фуд)",0.065145
270,(category_Фаст Фуд),"(category_Рестораны, category_Транспорт)",0.064828
...,...,...,...
131,(category_Финансовые услуги),"(category_Аптеки, category_Одежда/Обувь)",0.009897
161,(category_Финансовые услуги),"(category_Аптеки, category_Топливо)",0.008605
101,(category_Финансовые услуги),"(category_Аптеки, category_Дом/Ремонт)",0.008255
221,(category_Финансовые услуги),"(category_Наличные, category_Фаст Фуд)",0.007981


In [84]:
 #sort_by_metric('conviction', 0.8)

AttributeError: 'float' object has no attribute 'shape'

<h2> Category advice
  

In [85]:
merchants_per_group = 5

personal_categories = raw_transactions[['party_rk', 'account_rk', 'category', 'merchant_type', 'merchant_group_rk', 'transaction_amt_rur',]].copy()
personal_categories = raw_transactions.groupby(['party_rk', 'account_rk', 'category', 'merchant_group_rk']).count()
personal_categories.reset_index()
personal_categories.groupby(['party_rk', 'account_rk', 'category', 'merchant_group_rk']).nlargest(5, columns=['transaction_dttm'])

AttributeError: 'DataFrameGroupBy' object has no attribute 'nlargest'